# Einsendeaufgabe 1: Numerische Genauigkeit und Gleichungssysteme (100 Punkte)
In dieser Aufgabe sollen Sie ein wenig mehr Erfahrung mit NumPy und numerischen Methoden gewinnen.  
Zur Erinnerung empfehle ich an dieser Stelle, die Definition der [IEEE-Fließkommazahlen](https://de.wikipedia.org/wiki/IEEE_754) zu wiederholen.  

## Addition von Zahlen (20 Punkte) 

Gegeben sei ein Array *array*, dass 100 mal die Zahl $10^{-16}$ enthält und einmal (als ersten Eintrag) die Zahl $1$. 

In [ ]:
import numpy as np

array = np.concatenate(([1], np.full(100, 1e-16)))
print(array)

**Aufgabe:** Addieren Sie alle Werte in `array`:
- mit der Funktion `np.sum()`
- mit einer Schleife, die von **vorne nach hinten** über `array` iteriert
- mit einer Schleife, die von **hinten nach vorne** über `array` iteriert

_Points:_ 6

In [ ]:
sum_numpy = np.sum(array)
print(f"{sum_numpy=}")

sum_forward = 0.0
for x in array:
    sum_forward += x
print(f"{sum_forward=}")

sum_reverse = 0.0
for x in reversed(array):
    sum_reverse += x
print(f"{sum_reverse=}")

<!-- BEGIN QUESTION -->

**Aufgabe:** Erklären Sie die Ergebnisse. Wie werden die Zahlen 1 und 1e-17 im Computer dargestellt? (ausführliche Erläuterung erforderlich!)  

_Points:_ 12

<span style="color:blue">

<h2>Numerische Genauigkeit von verschiedenen Additionsverfahren</h2>

Je nach Methode ergibt sich ein anderes Additionsergebnis:

```
sum_numpy   = np.float64(1.0000000000000084)
sum_forward = np.float64(1.0)
sum_reverse = np.float64(1.00000000000001)
```

---

Das liegt im Wesentlichen an der begrenzten bzw. endlichen Gleitkomma-Darstellung mithilfe des Computers, die zu diesen numerischen Ungenauigkeiten führt.

**[`np.float64`](https://numpy.org/doc/2.1/reference/arrays.scalars.html#numpy.float64)** ist eine 64-bit-Gleitkommazahl. Die Zahl wird intern gemäß dem IEEE-754-Standard wie folgt dargestellt:

- **1 Bit** für das Vorzeichen ($0$ = positiv, $1$ = negativ)
- **11 Bits** für den Exponenten
- **52 Bits** für die Mantisse

Das ergibt insgesamt: $1 + 11 + 52 = 64$ Bits.

Die kleinste Zahl, die man zu `1.0` addieren kann, sodass das Ergebnis sich von `1.0` unterscheidet, nennt man die **Maschinengenauigkeit** oder **machine epsilon**. Das ist der kleinste Unterschied, den ein `np.float64` von `1.0` noch unterscheiden kann – und genau hier zeigen sich die Grenzen der numerischen Genauigkeit bei Gleitkommazahlen.

```python
np.finfo(float).eps  
# np.float64(2.220446049250313e-16)
```

---

Die $100$ gleich großen Werte ($=10^{-16}$) liegen alle unterhalb von $\text{eps}$.  Zum Beispiel gilt: $1.0 + 10^{-16} = 1.0$,  weil $10^{-16} < \text{eps}$ – also zu klein ist, um überhaupt eine Änderung in der Darstellung von $1.0$ zu bewirken.

Die Summanden **„verpuffen“** numerisch. Das Resultat bleibt exakt **$1.0$**.

```python
print(1e-16 < np.finfo(float).eps)     # True
print(1.0 + 1e-16 == 1.0)              # True
```

---

Der **Bias** ist die Exponentenverschiebung, die verwendet wird, um den Exponenten nicht als vorzeichenbehaftete Zahl zu speichern.  
Exponenten können so als positive Zahlen **ohne Vorzeichenbit** abgelegt werden – das vereinfacht die Verarbeitung durch die Hardware.  
Anstatt z. B. den echten Exponenten $-5$ zu speichern, speichert man stattdessen $-5 + \text{Bias}$.

---

Mit $11$ Bits für den Exponenten können Zahlen von $0$ bis $2^{11} - 1 = 2047$ dargestellt werden.  
Der Bias für **np.float64** beträgt:  
$\text{Bias} = 2^{11 - 1} - 1 = 1023$  
Das entspricht der Mitte des 11-Bit-Bereichs.  

---

Die Werte $0$ und $2047$ sind reserviert für Sonderfälle (z. B. Null, Inf, NaN).  
So entsteht eine **einheitliche Rechenlogik**, z. B. auch für die Addition.

---

**Beispiel:**  
Mathematisch gilt: $1.0 = 1.0 \cdot 2^0$ → echter Exponent $= 0$  
In IEEE 754 wird gespeichert: $e = 0 + \text{Bias} = 1023_{10} = 01111111111_2$

Die **Mantisse** speichert nur die **Nachkommastellen**,  
die **Vorkommastelle $1$ ist implizit**.  
Die Mantisse ist 52 Bit lang → im Fall von $1.0$: alle Bits gleich 0.

**Darstellung in IEEE 754 (64 Bit) für 1.0:**


$1.0_{10} = 0 01111111111 000...000$

Oder mathematisch:

$1.0 = (-1)^s · 1.m · 2^{e - 1023}$

mit:  
- $s = 0$ (positiv)  
- $m = 0...0$  
- $e = 1023$

---

Genauso kann z. B. $1e{-17}$ als  
$$1e{-17} = (-1)^s \cdot 1.m \cdot 2^{e - 1023}$$  
dargestellt werden.  
Dabei ist $s = 0$.  
Für die Berechnung von Exponent und Mantisse benötigt man die **Binärdarstellung von $1e{-17}$** – die lässt sich mit Python oder NumPy bestimmen.

```python
np.frexp(1e-17) 
# (np.float64(0.7205759403792794), np.int32(-56)). 
# Das bedeuetet 1e-17 = 0.7205759403792794 * 2^(-56). 
# https://numpy.org/doc/2.2/reference/generated/numpy.frexp.html
```

Um die normalisierte Darstellung zu erhalten wird die Mantisse $0,7205759403792794$ mit $2$ multipliziert und die Basis mit dem Exponenten $2^{-56}$ durch $2$ dividiert. Dadurch ändert sich nichts am Zahlenwert. Somit ist $0.7205759403792794 \cdot 2^{-56}$ = $1,4411518807585588 \cdot 2^{-57}$.

Alternativ zur Benutzung von `frexp` kann man die Gleichung

$$1e{-17} = x \cdot 2^n$$

mit $1 \leq x < 2$ wie folgt lösen:

$$\log_2(1e{-17}) = \frac{\log_2(1e{-17})}{\log_{10}(2)} \approx -56,45$$

Es folgt:

$$n = -57$$

Danach kann die Gleichung

$$1e{-17} = x \cdot 2^{-57}$$

nach $x$ aufgelöst werden. Daraus folgt:

$$x = 2^{57} \cdot 1e{-17} = 1,44115188075855872$$

Also gilt:

$$1e{-17} = 1,44115188075855872 \cdot 2^{-57}$$

---

In IEEE 754 wird der Exponent als $-57 + 1023 = 966$ gespeichert, wobei:

$$966_{10} = 01111000110_2$$

Der Exponent wird als 11-Bit-Binärzahl gespeichert.

---

Nun zur Mantisse:  
IEEE speichert die Mantisse $0,44115188075855872$ ohne das führende $1$.  
Die Zahl wird iterativ mit 2 multipliziert, und das Ergebnis der 52 Multiplikationen wird von oben nach unten notiert, um die Mantisse zu berechnen.

Die ersten Schritte der Berechnung sind:

$$0,44115188075855872 \cdot 2 = 0,88230376151711744 \quad \rightarrow \quad 0$$

$$0,88230376151711744 \cdot 2 = 1,76460752303423488 \quad \rightarrow \quad 1$$

$$0,76460752303423488 \cdot 2 = 1,52921504606846976 \quad \rightarrow \quad 1$$

</span> 

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Aufgabe:** Welches Verfahren verwendet `np.sum()` für die Addition?

_Points:_ 2

<span style="color:blue">

<h2>Kaskadensummation</h2>

NumPy verwendet ein numerisch besseres Verfahren (partielle paarweise Summation), das in vielen Anwendungsfällen zu einer höheren Genauigkeit führt. In der numerischen Mathematik bezeichnet paarweise Summation – auch Kaskadensummation genannt – ein Verfahren zum Addieren von Gleitkommazahlen, das Rundungsfehler im Vergleich zur sequentiellen Summation deutlich reduziert.

https://numpy.org/doc/stable/reference/generated/numpy.sum.html

https://en.wikipedia.org/wiki/Pairwise_summation

</span>

<!-- END QUESTION -->

## Grenzwerte von Funktionen (20 Punkte)

Geben sei die Funktion $f(x)=\frac{e^x-1}{x}$

Es gilt: $lim_{x \to 0}f(x) = 1$

**Aufgabe:** Schreiben Sie eine Python Funktion `f(x)`, mit deren Hilfe Sie $f(x)$ für $x= 10^{-1}, 10^{-2}, \cdots , 10^{-15}$ mit NumPy berechnen und geben Sie das Ergebnis aus.

**Aufgabe:** Berechnen Sie nun die Funktion für  $x= 10^{-16}, 10^{-17}, \cdots , 10^{-20}$

_Points:_ 8

<span style="color:blue">
<h2>Berechnung der Funktion f(x) = (exp(x) - 1) / x </h2>
</span>


<span style="color:blue">
In dieser Aufgabe soll die Funktion f(x) für x = 10<sup>-1</sup>, 10<sup>-2</sup>, ..., 10<sup>-15</sup> berechnet werden. Danach soll die Funktion für noch kleinere Werte von x, nämlich x = 10<sup>-16</sup>, 10<sup>-17</sup>, ..., 10<sup>-20</sup>, berechnet werden. Der Grenzwert der Funktion für x → 0 ist 1.
</span>


In [ ]:
import numpy as np

# Definition der Funktion f(x) = (e^x - 1)/x
def f(x):
        return (np.exp(x) - 1) / x

# Erzeuge die Werte x = 10^(-1), 10^(-2), ..., 10^(-15)
input_numbers = np.array([10**(-i) for i in range(1, 16)])

result_limits = f(input_numbers)

#Ausgabe
for x_val, fx in zip(input_numbers, result_limits):
# Die Anwendung von zip gibt einen Iterator zurück, der in der Lage ist Tupel zu erzeugen
    print(f"x = {x_val:.0e}, f(x) = {fx}")

In [ ]:
input_numbers_2 = np.array([10**(-i) for i in range(16, 21)])
result_limits_2 = f(input_numbers_2)

#Ausgabe
for x_val, fx in zip(input_numbers_2, result_limits_2):
# Die Anwendung von zip gibt einen Iterator zurück, der in der Lage ist Tupel zu erzeugen
    print(f"x = {x_val:.0e}, f(x) = {fx}")

<!-- BEGIN QUESTION -->

**Aufgabe:** Was fällt auf? Welche Erklärung haben Sie für das Ergebnis? (Ausführliche Erklärung erforderlich) 

_Points:_ 12

<span style="color:blue">
<h2> Erklärung des Verhaltens der Funktion bei kleinen x-Werten </h2>

Für Werte von x zwischen `1e-01` bis `1e-14` liefert die Funktion erwartungsgemäß Ergebnisse nahe bei **1**. Ab etwa 1e-15 treten jedoch drastische Änderungen auf, die durch Rundungsfehler bei der Berechnung von `e^x - 1` bedingt sind. Besonders bei sehr kleinen x-Werten wird der Unterschied zwischen `e^x` und `1` auf dem Computer nicht mehr erkennbar, was zu fehlerhaften Ergebnissen führt. Eine stabilere Berechnung kann durch die Verwendung der Funktion `np.expm1(x)` von NumPy erreicht werden.


Von `1e-01` bis `1e-14` ergibt die Funktion erwartungsgemäß Werte, die sehr nahe bei **1** liegen.  
Doch ab etwa `1e-15` ändert sich das Verhalten drastisch:

x = 1e-15, f(x) = 1.1102230246251565   
x = 1e-16, f(x) = 0.0  
x = 1e-17, f(x) = 0.0  
x = 1e-18, f(x) = 0.0  
x = 1e-19, f(x) = 0.0  
x = 1e-20, f(x) = 0.0  

<h3> Erklärung: Rundungsfehler </h3>

Für sehr kleine `x` gilt näherungsweise:  
`exp(x) ≈ 1 + x`

Wenn `x` so klein wird, dass `e^x` und `1` auf dem Computer **nicht mehr unterscheidbar** sind, wird der Ausdruck `e^x - 1` numerisch zu **null**.

> Beispiel:  
> `x = 1e-17` → `exp(x) ≈ 1.00000000000000001`  
> Die Differenz `e^x - 1` ist kleiner als die **Maschinengenauigkeit** `ε` (Epsilon).  
> Deshalb wird `e^x - 1` vom Computer als **0.0** berechnet, da ein 64-Bit-Gleitkommawert (float64) **nicht mehr ausreichend genau** ist.

<h3> Lösung: Stabilere Berechnung mit NumPy </h3>

NumPy bietet mit `np.expm1(x)` eine spezielle Funktion an, die den Ausdruck `exp(x) - 1` **numerisch stabiler** berechnet – insbesondere für sehr kleine `x`.

</span>

In [ ]:
def f_stabil(x):
    return np.expm1(x) / x

input_numbers_3 = np.array([10**(-i) for i in range(10, 20)])
result_limits_3 = f_stabil(input_numbers_3)

#Ausgabe
for x_val, fx in zip(input_numbers_3, result_limits_3):
# Die Anwendung von zip gibt einen Iterator zurück, der in der Lage ist Tupel zu erzeugen
    print(f"x = {x_val:.0e}, f(x) = {fx}")

<!-- END QUESTION -->

## Gauß-Verfahren und LU-Zerlegung (60 Punkte) 
Obwohl man, wie wir sehen werden, lineare Gleichungssysteme einfach mit SciPy lösen kann, wollen wir in dieser Aufgabe  einmal selbst eine sogenannte LU-Zerlegung bzw. das Gauß-Verfahren implementieren. Außerdem werden wir diesen Code auch später noch verwenden, wenn wir uns mit Performance-Optimierung und Parallelisierung beschäftigen.

Das Gauß-Verfahren ist ein wichtiges Verfahren zum Lösen von linearen Gleichungssystemen, wir gehen hier schrittweise mit der Implementierung vor.

**Aufgabe:** Implementieren Sie die Funktion `b_solve` zur Lösung einer Dreiecksgleichung der Form:

$\left[ {\begin{array}{cccc} 
a_{11} & a_{12} & \cdots & a_{1n}\\ 
0 & a_{22} & \cdots & a_{2n}\\ 
\vdots & \vdots & \ddots & \vdots \\ 
0 & 0 & \cdots & a_{nn}\\ 
\end{array} } \right] \left[{\begin{array}{c}  
x_1\\ 
x_2\\ 
\vdots\\ 
x_n\\ 
\end{array}}\right] =  
\left[{\begin{array}{c}  
b_1\\ 
b_2\\ 
\vdots\\ 
b_n\\ 
\end{array}}\right]$ 

Hier beginnt man mit der Löung der letzten Zeile, denn es gilt:

$x_n = b_n/a_{nn} $
und dann

$x_{n-1} =  (b_{n-1}-a_{(n-1),n}*x_{n})/a_{(n-1),(n-1)}$

$x_{n-2} =  (b_{n-2}-(a_{(n-2),n}*x_{n}+a_{(n-2),(n-1)}*x_{n-1})/a_{(n-2),(n-2)}$


*Hinweis*: Achten Sie darauf, dass ihr Code nicht nur korrekt, sondern auch effizient ist. 

_Points:_ 12

<span style="color:blue">

  <h2>Backsubstitution-Verfahren</h2>

Im Folgenden wird eine Funktion zur Lösung eines linearen Gleichungssystems mit einer oberen Dreiecksmatrix implementiert. Dieses sogenannte Backsubstitution-Verfahren stellt einen zentralen Schritt im Rahmen des Gauß-Verfahrens bzw. der LU-Zerlegung dar. Dabei wird die Lösung des Systems zeilenweise von unten nach oben bestimmt. Die Implementierung legt besonderen Wert auf numerische Stabilität und Effizienz.
</span>

In [ ]:
import numpy as np

def b_solve(A, b):
    """Solve a system of linear equations A * x = b where A is an upper triangular matrix."""

    n = len(b)  # Anzahl der Variablen im Lösungsvektor b bzw. Länge des Vektors b
    x = np.zeros_like(b, dtype=float)  # Vektor für die Lösung, initialisiert mit Nullen
    
    # Rückwärtseinsetzung
    # Die arrays fangen mit Index 0 an
    # Iteration von der letzten Zeile bis zur ersten 
    # range(start, "bis zu aber nicht einschließlich" stop, step)
    for i in range(n - 1, -1, -1):  
        # Berechnung der Summe der Produkte der bekannten x-Werte
        summation = np.dot(A[i, i + 1:], x[i + 1:])
        x[i] = (b[i] - summation) / A[i, i]  # Berechnung des aktuellen x_i
        #print(f"x[{i}] = (b[{i}] - summation) / A[i, i] = ({b[i]} - {np.dot(A[i, i + 1:], x[i + 1:])}) / {A[i,i]} = {x[i]}")
    return x

#Beispiel:
A = np.array([[2,3,5,1],[0,4,6,2],[0,0,7,3],[0,0,0,8]])
b = np.array([9,8,7,8])
x = b_solve(A,b)
#print(f"{np.dot(A,x)} == {b}") #"[9. 8. 7. 8.] == [9 8 7 8]"

**Aufgabe:** Implementieren Sie die LU-Zerlegung einer Matrix mit Pivotisierung.  
Die Funktion `lu_decomposition` soll die Matrizen L und U berechnen.  
Zu Beginn wird der Funktion eine Matrix A übergeben.  


Implementieren Sie die Funktion "in-place", d.h. die Ergebnisse für L und U werden direkt in A geschrieben.  
Dabei wird die ursprüngliche Matrix überschrieben.  
Die Funktion gibt den Pivot-Vektor zurück, der die folgende Form haben sollte:

$ 
P=\left[{\begin{array}{ccc} 
0 & 1& 0\\ 
0 & 0& 1\\ 
1 & 0& 0\\ 
\end{array} } \right] \Rightarrow \left[{\begin{array}{c}  
1\\ 
2\\ 
0\\ 
\end{array}}\right] 
$ 

D.h. es wird nur der Index des Wertes in jeder Zeile gespeichert, der nicht Null ist.

Wenn man dies macht, kann man später für die Lösung einfach den Vektor `bc = b[P]` verwenden, um auch $b$ richtig zu pivotisieren.  

Hinweise: 
- Die Pivot-Zeile lässt sich am einfachsten mit `np.argmax` finden. Die Zeile mit dem höchsten Betrag können Sie mit `np.abs` finden.

- Tauschen zweier Elemente in einem NumPy Array: `v[[a, b]] = v[[b, a]]`

- Versuchen Sie, die interne Vektorisierung von NumPy im Eliminationsschritt zu nutzen. Verwenden Sie dazu die Slicing-Syntax, um die innerste Schleife zu ersetzen.

_Points:_ 15

<span style="color: blue;">
  <h2>LU-Zerlegung mit Pivotisierung</h2>
</span>

<span style="color: blue;">
Im folgenden Code wird die LU-Zerlegung einer Matrix A mit Pivotisierung implementiert. Die Matrix A wird in die Form A = LU zerlegt, wobei L und U die untere und obere Dreiecksmatrix sind. Zusätzlich wird ein Pivot-Vektor zurückgegeben, der die Zeilenvertauschung anzeigt. Die Berechnung erfolgt in-place und nutzt NumPy für effiziente Vektorisierung.
</span>

In [ ]:
def lu_decomposition(A):
    """
    Implement LU-Decomposition of a square matrix.
    This function works in-place i.e. it stores information about L and U in A and returns the pivot vector.
    """
    ...
    return p

<!-- BEGIN QUESTION -->

**Aufgabe:** Schreiben Sie eine Funktion `custom_solve`, welche mit Hilfe von `lu_decomposition` ein lineares Gleichungssystem der Form $A*x=b$ löst.  

Hinweis: Versuchen Sie, die interne Vektorisierung von NumPy zu nutzen. Verwenden Sie dazu die Slicing-Syntax, um die innerste Schleife zu ersetzen.

_Points:_ 12

In [ ]:
def custom_solve(A, b):
    """Solve a system of linear equations A * x = b and return result vector x."""
    ...
    return x

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Aufgabe:** Schreiben Sie eine Testfunktion, die überprüft, ob die Lösung ihrer Implementierung des Gauß-Verfahrens richtig ist.  
Testen Sie Gleichungssysteme der Größe $16 \times 16$, $32\times 32$ und $64 \times 64$.

Sie können zum Vergleich die SciPy Funktion `linalg.solve` verwenden:

_Points:_ 6

In [ ]:
# Beispiel linalg
import scipy.linalg as linalg

n = 128
rng = np.random.default_rng(seed=123)
A = rng.random((n, n))
b = rng.random(n)
c = linalg.solve(A, b)

In [ ]:
import scipy.linalg as linalg

# test different dimensions
...

<!-- END QUESTION -->

**Aufgabe:** Wir wollen nun weiter die *fertigen* SciPy Funktionen betrachten. Mit den Funktionen `lu` und `lu_factor` aus dem Packet `scipy.linalg `  können Sie eine LU-Dekomposition durchführen [[1]](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu_factor.html#scipy.linalg.lu_factor) und [[2]](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu.html).  
Beide Funktionen haben unter anderem den Parameter `overwrite_a`, der `False` oder `True` sein kann. 

Evaluieren Sie die Performance (Laufzeit) der LU-Zerlegung einer 4096x4096 Matrix, wenn Sie jeweils die Funktionen `lu_factor` und `lu` mit `overwrite_a = True` oder `overwrite_a = False` verwenden.

In der Liste `minimums` sollen die minimalen Laufzeiten in folgender Reihenfolge stehen:
1. `lu_factor` mit `overwrite_a = False`
2. `lu_factor` mit `overwrite_a = True`
3. `lu` mit `overwrite_a = False`
4. `lu` mit `overwrite_a = True`

Hinweis: Da die Matrix A bei der Verwendung von `overwrite_a = True` verändert wird, kann es zu einer Vereinfachung der Berechnung kommen, wenn die Funktion mehrfach nacheinander ausgeführt wird. Aus diesem Grund muss die `timeit` Funktion [repeat](https://docs.python.org/3/library/timeit.html#timeit.repeat) mit `repeat = 2` und `number = 1`  verwendet werden. Die Matrix wird im `setup` Parameter erstellt (aus Zufallswerten mit dem vorgegebenen "random number generator").

_Points:_ 6

In [ ]:
from scipy.linalg import lu_factor, lu
from timeit import repeat

n = 8192
number = 1
repetitions = 2
# store all four minimal runtimes in this list
minimums = []
rng = np.random.default_rng(seed=123)

...

**Aufgabe:** Untersuchen Sie den Speicherverbrauch der 4 Varianten mit memray.

Um den Einfluss der anderen Berechnungen zu minimieren, empfehlen wir hier, die Berechnungen in eine neue Datei zu schreiben und mit `!` in einer Code-Zelle memray's Command-Line Interface zu verwenden. 

Am besten tracen Sie nur die LU Funktion mit einem Context Manager!
Verwenden Sie jeweils folgende Dateinamen als `file_name` Parameter in `memray.Tracker`:

- lu_inplace.bin
- lu_notinplace.bin
- lu_factor_inplace.bin
- lu_factor_notinplace.bin

Beantworten Sie bitte die folgenden Fragen:
* Welche Version verwendet am meisten Speicher? 
* Bei welcher Version ist der Speicherverbrauch am konstantesten? 

_Points:_ 9

In [ ]:
%%file lu_inplace.py
import memray
import numpy as np
from scipy.linalg import lu

n = 8192
rng = np.random.default_rng(seed=123)
...

In [ ]:
%%file lu_notinplace.py
import memray
import numpy as np
from scipy.linalg import lu

n = 8192
rng = np.random.default_rng(seed=123)
...

In [ ]:
%%file lu_factor_inplace.py
import memray
import numpy as np
from scipy.linalg import lu_factor

n = 8192
rng = np.random.default_rng(seed=123)
...

In [ ]:
%%file lu_factor_notinplace.py
import memray
import numpy as np
from scipy.linalg import lu_factor

n = 8192
rng = np.random.default_rng(seed=123)
...

In [ ]:
...

In [ ]:
#!rm -rf lu_* memray-*